### Effective Public School 

source: https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html 

variables: ALL_RLA03numvalid_1819, ALL_RLA03pctprof_1819, ALL_RLA08numvalid_1819, ALL_RLA08pctprof_1819, ST_LEAID, LEAID, LEANM, FIPST, STNAM

numvalid    
The number of students who completed the state assessment and for whom a proficiency level was assigned

pctprof    
The percentage of students scoring at or above the state’s proficiency level on the assessment



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import  drive
    drive.mount('/drive')
    data_path = '/drive/Shared drives/Capstone/notebooks/data'
else:
    data_path = 'data'

Mounted at /drive


In [ ]:
# crosswalk csv here https://exhibits.stanford.edu/data/catalog/db586ns4974
# seda_crosswalk_4.1.csv (june 2021)
# link to documentation: https://stacks.stanford.edu/file/druid:db586ns4974/seda_documentation_4.1.pdf
# Data files : https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html


In [ ]:
# compare grades 3 - 8 in 2018-2019 
crosswalk_df = pd.read_csv(f'{data_path}/raw/seda_crosswalk_4.1.csv', dtype={'FIPS': str})
merged_df = crosswalk_df[['sedacounty', 'leaid']].rename(columns={'leaid': 'LEAID', 'sedacounty': 'FIPS'})

latest_df = pd.read_csv(f'{data_path}/raw/rla-achievement-lea-sy2018-19-wide.csv')
# latest url csv: 
# latest_df = pd.read_csv('https://www2.ed.gov/about/inits/ed/edfacts/data-files/rla-achievement-lea-sy2018-19-wide.csv')

merged_df = merged_df.merge(latest_df, how="left", left_on='LEAID', right_on='LEAID')
merged_df = merged_df.drop_duplicates()
merged_df.columns = merged_df.columns.str.upper()
merged_df.to_csv(f'{data_path}/interim/fips_lastest_edu_prof_all.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (18,20,22,26,36,38,40,42,44,46,50,52,54,98,108,112,114,122,124,126,128,130,136,138,140,144,150,152,154,156,158,162,170,180,182,184,192,194,236,240,248,250,254,256,258,260) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# number of valid tests taken for each grade can be used for weighting 
num_valid = [f'ALL_RLA0{i}NUMVALID_1819' for i in range(3, 9)]
num_valid


['ALL_RLA03NUMVALID_1819',
 'ALL_RLA04NUMVALID_1819',
 'ALL_RLA05NUMVALID_1819',
 'ALL_RLA06NUMVALID_1819',
 'ALL_RLA07NUMVALID_1819',
 'ALL_RLA08NUMVALID_1819']

## Processing differences in percent proficient from grade 3 - 8: 
- if the numbers were surpressed to protect student privacy, the mid range number was chosen for ranges less than or equal to 10. 
- if the range was more than 10, that number was not used 
- if the percent was PS , it was replaced with NAN and not included in the calculation of difference
- yearly average difference was calculated across grades 03 - 05 for either longitudinal data or latest for year 2018-2019. If there are mulitple averages, we took the average of those for the final diff

'LE20'

In [ ]:

latest_cats = [
          'ALL_RLA03PCTPROF_1819',	
          'ALL_RLA04PCTPROF_1819',
          'ALL_RLA05PCTPROF_1819',
          'ALL_RLA06PCTPROF_1819',
          'ALL_RLA07PCTPROF_1819',
          'ALL_RLA08PCTPROF_1819'
          ]

def range_to_median(row, all_cats=latest_cats):
    # set to NAN for values that have higher range than 10
    # GT greater than, GE greater than or equal to
    # LT less than, LT less than or equal to
    mapped_strings = {
                 'LE5': 2.5,
                 'LE10': 5,
                 'LE20': np.nan,
                 'LT50': np.nan,
                 'GE50': np.nan,
                 'GE80': np.nan,
                 'GE90': 95,
                 'GE95': 97.5,
                 'GE99': 99.5,
                 'PS':np.nan,
                 ".": np.nan
    }
    diffs = []

    for i, cat in enumerate(all_cats):
      if row[cat] in mapped_strings.keys():
        row[cat] = mapped_strings[row[cat]]
      else:
        try:
          
          row[cat] = float(row[cat])
        except:
         
          row[cat] = (int(row[cat].split('-')[0]) + int(row[cat].split('-')[1]))/2
          
    return row


# Function to get difference between grades
# including "scores" and "diffs" just for reference 
def get_diff(row, all_cats=latest_cats):
    diffs = []
    scores = []
    for i, cat in enumerate(all_cats):
      if np.isnan(row[all_cats[i]]):
        scores.append('NA')
        continue
      else:
        scores.append(row[all_cats[i]])
        try:
          if np.isnan(row[all_cats[i+1]]):
            
            continue
          else:
            diffs.append(row[all_cats[i+1]] - row[all_cats[i]])
            
        except:
          continue
    if diffs:
      row['diff'] = sum(diffs)/len(diffs)
      # used this to verify 
      row['diffs'] = diffs
      row['scores'] = scores
    else:
      row['diff'] = 0
      row['diffs'] = []
      row['scores'] = []
    return row

merged_df = merged_df[['FIPS','LEANM']+num_valid+latest_cats]
# drop empty fips rows 
merged_df = merged_df.dropna(subset=['FIPS'])
# replace ps with previous column
cleaned_scores_df = merged_df.apply(range_to_median, axis=1)
# remove nan rows for  
diff_df = cleaned_scores_df.apply(get_diff, axis=1)


In [ ]:
# remove some bad data and replace with NAN
diff_df.replace('.', np.nan, inplace=True)

In [ ]:
# make sure they are all numerical values so we can sum them
diff_df[num_valid] = diff_df[num_valid].apply(pd.to_numeric)

In [ ]:
# sum the total number of valid test takers for grades 3 - 8 to act as the weight
diff_df['weight'] = diff_df[num_valid].sum(axis=1)

In [ ]:
diff_df.sample(10)

,FIPS,LEANM,ALL_RLA03NUMVALID_1819,ALL_RLA04NUMVALID_1819,ALL_RLA05NUMVALID_1819,ALL_RLA06NUMVALID_1819,ALL_RLA07NUMVALID_1819,ALL_RLA08NUMVALID_1819,ALL_RLA03PCTPROF_1819,ALL_RLA04PCTPROF_1819,ALL_RLA05PCTPROF_1819,ALL_RLA06PCTPROF_1819,ALL_RLA07PCTPROF_1819,ALL_RLA08PCTPROF_1819,diff,diffs,scores,weight
35699,13295.0,Chickamauga City,80.0,93.0,97.0,110.0,99.0,76.0,77.0,62.0,67.0,57.0,62.0,52.0,-5.0,"[-15.0, 5.0, -10.0, 5.0, -10.0]","[77.0, 62.0, 67.0, 57.0, 62.0, 52.0]",555.0
113237,44007.0,Smithfield,177.0,162.0,183.0,220.0,185.0,208.0,62.0,62.0,62.0,52.0,47.0,57.0,-1.0,"[0.0, 0.0, -10.0, -5.0, 10.0]","[62.0, 62.0, 62.0, 52.0, 47.0, 57.0]",1135.0
8380,6107.0,Farmersville Unified,182.0,188.0,188.0,205.0,183.0,213.0,27.0,22.0,27.0,22.0,17.0,22.0,-1.0,"[-5.0, 5.0, -5.0, -5.0, 5.0]","[27.0, 22.0, 27.0, 22.0, 17.0, 22.0]",1159.0
40538,17067.0,Carthage ESD 317,56.0,44.0,45.0,55.0,42.0,37.0,54.5,44.5,44.5,44.5,74.5,54.5,0.0,"[-10.0, 0.0, 0.0, 30.0, -20.0]","[54.5, 44.5, 44.5, 44.5, 74.5, 54.5]",279.0
47881,18067.0,Kokomo School Corporation,376.0,396.0,412.0,393.0,392.0,383.0,32.0,33.0,35.0,29.0,36.0,31.0,-0.2,"[1.0, 2.0, -6.0, 7.0, -5.0]","[32.0, 33.0, 35.0, 29.0, 36.0, 31.0]",2352.0
81775,34025.0,Eatontown Public School District,94.0,96.0,97.0,104.0,110.0,104.0,62.0,67.0,57.0,72.0,67.0,67.0,1.0,"[5.0, -10.0, 15.0, -5.0, 0.0]","[62.0, 67.0, 57.0, 72.0, 67.0, 67.0]",605.0
114846,45079.0,Richland 02,2116.0,2170.0,2213.0,2320.0,2247.0,2180.0,51.0,53.0,44.0,45.0,47.0,48.0,-0.6,"[2.0, -9.0, 1.0, 2.0, 1.0]","[51.0, 53.0, 44.0, 45.0, 47.0, 48.0]",13246.0
48231,18129.0,M S D Mount Vernon,152.0,125.0,140.0,135.0,165.0,153.0,67.0,57.0,52.0,52.0,47.0,52.0,-3.0,"[-10.0, -5.0, 0.0, -5.0, 5.0]","[67.0, 57.0, 52.0, 52.0, 47.0, 52.0]",870.0
138136,53031.0,Chimacum School District,53.0,68.0,71.0,55.0,60.0,49.0,54.5,62.0,52.0,44.5,44.5,44.5,-2.0,"[7.5, -10.0, -7.5, 0.0, 0.0]","[54.5, 62.0, 52.0, 44.5, 44.5, 44.5]",356.0
607,1043.0,Cullman City,249.0,273.0,228.0,265.0,244.0,252.0,62.0,67.0,62.0,67.0,62.0,67.0,1.0,"[5.0, -5.0, 5.0, -5.0, 5.0]","[62.0, 67.0, 62.0, 67.0, 62.0, 67.0]",1511.0


In [ ]:
# Create weighted average
# note: weights are the total number of students who took the test, grades 3-8

def my_agg(x):
    names = {'weighted_av': (x['weight'] * x['diff']).sum()/x['weight'].sum()}
    return pd.Series(names, index=['weighted_av'])

In [ ]:
grouped_df = diff_df.groupby('FIPS').apply(my_agg).reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [ ]:
# rename column to fit with existing workflow
grouped_df.rename(columns={'weighted_av': 'avg_edu_prof_diff'}, inplace=True)


In [ ]:
grouped_df.head()

,FIPS,avg_edu_prof_diff
0,1001.0,-2.000000
1,1003.0,-0.800000
2,1005.0,0.817457
3,1007.0,-1.000000
4,1009.0,-0.491528


In [ ]:
# grouped_df['avg_edu_prof_diff'] = grouped_df['weighted_av'].div(100)
grouped_df = grouped_df[['FIPS', 'avg_edu_prof_diff']]

In [ ]:
grouped_df.head()

,FIPS,avg_edu_prof_diff
0,1001.0,-2.000000
1,1003.0,-0.800000
2,1005.0,0.817457
3,1007.0,-1.000000
4,1009.0,-0.491528


In [ ]:
grouped_df.to_csv(f'{data_path}/processed/avg_edu_prof_all.csv', index=False)

## Homeless enrolled for 2018 - 2019
https://www2.ed.gov/about/inits/ed/edfacts/data-files/school-status-data.html 


Notes:     
- Crosswalk CSV for year 2018 used to map FIPS codes to LEAID:
    - https://exhibits.stanford.edu/data/catalog/db586ns4974
    - filname: seda_crosswalk_4.1.csv (june 2021)
- Dataset for homeless student numbers is the unduplicated number of homeless students enrolled in each LEA at any time during the school year. 
    - https://www2.ed.gov/about/inits/ed/edfacts/data-files/lea-homeless-enrolled-sy2018-19-wide.csv

Methodology and Assumptions:
 - Puerto Rico was not broken down at a county level, all of Puerto Rico is under FIPS code 72129
 - Some Local Education Agencies (LEA) overlap, for example: Hoover City is in both Shelby County and Jefferson County. Because we have no way to divide the number of students in Hoover City properly, we added that number to both counties. 
 - To verify state totals those numbers can been seen https://nces.ed.gov/programs/digest/d19/tables/xls/tabn204.75d.xls 


In [ ]:
import pandas as pd
def get_homeless_students_df(recreate=False):
  """
  return a merged df crosswalk df and homeless student numbers df
  """

  if recreate: 
    crosswalk_df = pd.read_csv(f'{data_path}/raw/seda_crosswalk_4.1.csv')
    crosswalk_df = crosswalk_df[crosswalk_df['year']==2018]
    # Rename Columns for readability
    merged_df = crosswalk_df[['sedacounty','leaid']].rename(columns={'leaid': 'LEAID', 'sedacounty': 'FIPS', 'sedaschname': 'NAME'})
    
    df = pd.read_csv(f'{data_path}/raw/lea-homeless-enrolled-sy2018-19-wide.csv')

    # read csv from URL 
    # df = pd.read_csv('https://www2.ed.gov/about/inits/ed/edfacts/data-files/lea-homeless-enrolled-sy2018-19-wide.csv')

    # Only return certain columns
    df = df[['LEAID', 'TOTAL','STNAM']]

    # replace S with 1 student count between 0 and 2 is S
    df['TOTAL'] = df['TOTAL'].replace(['S'], 1).fillna(0).apply(pd.to_numeric, errors="coerce").fillna(0)

    # Merge on the LEAID Local Education Agency ID 
    merged_df = df.merge(merged_df, how="left", left_on='LEAID', right_on='LEAID')

    # merge all of Puerto Rico into one FIPS as a placeholder because it's not broken down 
    # By County 
    merged_df.loc[merged_df['STNAM']=='PUERTO RICO', 'FIPS'] = 72129.0



    # fill in any NAN totals with 0
    merged_df['TOTAL'].fillna(0, inplace=True)
    # Remove Duplicate rows

    merged_df = merged_df.drop_duplicates()

    # We only need totals for FIPS
    merged_df = merged_df[['FIPS', 'TOTAL', 'STNAM']]

    # Merge totals back with merged_df to get state name and verify state totals 
    totals = merged_df.groupby('FIPS')['TOTAL'].sum()
    totals_merged_df = merged_df[['FIPS', 'STNAM']].merge(totals, how="left", left_on='FIPS', right_on='FIPS')
    totals_merged_df = totals_merged_df.drop_duplicates()
    totals_merged_df = totals_merged_df.dropna()
    totals_merged_df = totals_merged_df.rename(columns={'TOTAL': 'HOM_STUDENTS'})
    totals_merged_df = totals_merged_df[['FIPS', 'HOM_STUDENTS']]
    totals_merged_df.to_csv(f'{data_path}/processed/homeless_students_201819-totals.csv', index=False)
  else:
    totals_merged_df = pd.read_csv(f'{data_path}/processed/homeless_students_201819-totals.csv')
  return totals_merged_df



In [ ]:
hom_df = get_homeless_students_df(recreate=False)

In [ ]:
hom_df.head()

,FIPS,HOM_STUDENTS
0,1095.0,499.0
1,1073.0,1291.0
2,1117.0,353.0
3,1089.0,495.0
4,1123.0,22.0


In [ ]:
#sanity check, verify the totals are valid for the fips
print(sum(list(hom_df[hom_df['FIPS']==51600]['HOM_STUDENTS'])))
hom_df[hom_df['FIPS']==51600]

2467.0


,FIPS,HOM_STUDENTS
2551,51600.0,2467.0


In [ ]:
hom_df.head()

,FIPS,TOTAL,STNAM
0,1095.0,101.0,ALABAMA
3,1095.0,196.0,ALABAMA
12,1073.0,51.0,ALABAMA
13,1117.0,51.0,ALABAMA
26,1089.0,60.0,ALABAMA


In [ ]:
# Verify Alabamas numbers, should be a bit more than 15k (check state totals)
# https://docs.google.com/spreadsheets/d/1ibXA_vwpFg_-AxpmlLPtt6YlZhpy8Lr5/edit#gid=577361330

print(sum(list(hom_df[hom_df['STNAM']=='ALABAMA']['TOTAL'])))

16189.0
